In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os, sys, scipy, pandas as pd, importlib

sys.path.append('/Users/alexisdevitre/Documents/code/hts-irradiation/')
import hts_fitting as hts, hts_misc as misc, hts_anneal as anneal, hts_showcase as sc
importlib.reload(anneal)

def get_fluence_steps(fpath):
    return pd.read_excel(fpath, sheet_name='fluence_steps', usecols=[0, 1, 2], names=['step', 'step_fluence', 'cumulated_fluence'])

def compute_tc_degradation(input_path='input/Tc_degradation.xlsx'):
    input_data = pd.read_excel(input_path, sheet_name='input')
    fpaths = []
    for tape in input_data.tape:
        tc, fl, magnet, fpaths, fluences = [], [], [], [], get_fluence_steps([tape+'/fluence/'+f for f in os.listdir(tape+'/fluence/') if 'Proton' in f][0])

        for i, row in input_data[input_data.tape == tape].iterrows():
            try:
                path = '{}/Tc/infield/{}'.format(tape, row.fname)
                tc.append(hts.fitTcMeasurement(path, wsz=row.smoothing_window, fit_range=(row.lowerbound_K, row.upperbound_K)))
                fl.append(fluences.cumulated_fluence.loc[fluences.step == row.step].values[0])

                if 'nomagnet' in path:
                    magnet.append(False)
                else:
                    magnet.append(True)

                fpaths.append(path)

            except RuntimeError as e:
                print(path, '\n\n', e)
            except ValueError as e:
                print(path, '\n\n', e)

        df = pd.DataFrame({'fpath': fpaths, 'cumulated_fluence': fl, 'magnet': magnet, 'tc': tc})
        with pd.ExcelWriter('output/{}-tc_degradation.xlsx'.format(tape), mode='w', engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='all_data', index=False)

def compute_ic_degradation(tapes, bins=None, function='powerLaw'):
    
    fluence = get_fluence_steps([tape+'/fluence/'+f for f in os.listdir(tape+'/fluence/') if 'Proton' in f][0])
    
    steps = [str(f) for f in np.sort(os.listdir(tape+'/Ic/infield')) if 'step' in f]
    
    data = None
    for step in steps:
        for magnet in ['magnet', 'no magnet']:
            path = tape + '/Ic/infield/' + step + '/{}/'.format(magnet)
            df = hts.getIcT([path+f for f in np.sort(os.listdir(path)) if 'Ic_' in f], function=function)[2]
            df['cumulated_fluence'] = fluence.cumulated_fluence.loc[fluence.step == step].values[0]
            
            if magnet == 'magnet':
                df['magnet'] = True
            else:
                df['magnet'] = False
                
            if data is not None:
                data = pd.concat([data, df], ignore_index=True)
            else:
                data = df
                
    with pd.ExcelWriter('output/{}-ic_degradation.xlsx'.format(tape), mode='w', engine='openpyxl') as writer:
        data.to_excel(writer, sheet_name='all_data', index=False)

    # aggregate data
    if bins is None:
        data['temperature_group'] = pd.cut(data['temperature'], bins=np.arange(0.3, 93.6, 0.2), right=False)
    else:
        data['temperature_group'] = pd.cut(data['temperature'], bins=bins, right=False)

    data = data.drop('fpath', axis=1).groupby(['temperature_group', 'cumulated_fluence', 'magnet']).agg(['mean', 'std', 'count']).dropna()
    data.columns = ['_'.join(col) for col in data.columns]
    data.drop(['temperature_count', 'ic_count', 'n_count', 'timeends_mean', 'timeends_std'], axis=1, inplace=True)
    data.rename(columns={'timeends_count': 'file_count'}, inplace=True)
    with pd.ExcelWriter('output/{}-ic_degradation.xlsx'.format(tape), mode='a', engine='openpyxl', if_sheet_exists='new') as writer:
        data.reset_index().to_excel(writer, sheet_name='aggregates', index=False)


In [7]:
compute_ic_degradation('ext_ffj51')

/Users/alexisdevitre/Documents/code/hts-irradiation/hts_fitfunctions.py:38: RuntimeWarning: invalid value encountered in power
  return 2e-7*(i/ic)**n
/Users/alexisdevitre/Documents/code/hts-irradiation/hts_fitfunctions.py:38: RuntimeWarning: invalid value encountered in power
  return 2e-7*(i/ic)**n
/Users/alexisdevitre/Documents/code/hts-irradiation/hts_fitting.py:317: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(ff.linear, ccut, vcut)
/Users/alexisdevitre/Documents/code/hts-irradiation/hts_fitting.py:317: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(ff.linear, ccut, vcut)
/var/folders/jl/4_5mv35x3wb9cccz5qcc6vmc0000gn/T/ipykernel_94421/1833376415.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
 